In [13]:
import numpy as np
from tkinter import *
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
from time import strftime
from datetime import datetime
import mysql.connector
import os
import cv2

In [14]:
class FaceReconizerSystem:
    def __init__(self, root):
        self.root = root
        self.root.geometry('1370x790+0+0')
        self.root.title('Face Reconigation System')
        
        # Putting title
        title_lbl = Label(self.root,text='FACE RECOGNITION',font=('times new roman',30,'bold'),bg='red',fg='white')
        title_lbl.place(x=0, y=0, width=1370, height=50)
        
        # Image add in LEFT
        img_left =Image.open('img/face_detector1.jpg') 
        img_left = img_left.resize((650,600), Image.ANTIALIAS)
        self.photoimg_left = ImageTk.PhotoImage(img_left)
        
        f_lbl = Label(self.root, image = self.photoimg_left)        
        f_lbl.place(x=0,y=55,width=650, height=600)
        
        # Image add in RIGHT
        img_right =Image.open('img/facial_recognition_system.jpg') 
        img_right = img_right.resize((750,600), Image.ANTIALIAS)
        self.photoimg_right = ImageTk.PhotoImage(img_right)
        
        f_lbl = Label(self.root, image = self.photoimg_right)        
        f_lbl.place(x=650,y=55,width=750, height=600)
        
        # buttom
        b1_1= Button(f_lbl,text='Face Recognition',cursor='hand2', command=self.face_recog,font=('times new roman',18,'bold'),bg='darkgreen',fg='white')
        b1_1.place(x=260, y=525, width=220, height=40)
        
    ###################### Attendance ##########################3
    
    def mark_attendance(self,i,r,n,d):
        with open("studentattendance.csv","r+",newline="\n") as f:
            myDataList=f.readlines()
            name_list=[]
            for line in myDataList:
                entry=line.split((","))
                name_list.append(entry[0])
            if((i not in name_list) and (r not in name_list) and (n not in name_list) and (d not in name_list)):
                now=datetime.now()
                d1=now.strftime("%d/%m/%Y") 
                dtString=now.strftime("%H:%M:%S")
                f.writelines(f"{i},{r},{n},{d},{dtString},{d1},Present \n")
            
    ################################## Face Recognition#############################
    def face_recog(self):
        def draw_boundry(img, classifier, scaleFactor, minNeighbors, color, text, clf):
            gary_image=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            features=classifier.detectMultiScale(gary_image, scaleFactor, minNeighbors)
            
            coord=[]
            for (x,y,w,h) in features:
                cv2.rectangle(img,(x,y),(x+w,y+h), (0,255,0),2)
                id,prediction= clf.predict(gary_image[y:y+h, x:x+w])
                confidence=int((100*(1-prediction/300)))
                
                conn=mysql.connector.connect(host='localhost', username='root', password='Aryanabhi@10', database='face_reconization')
                my_cursor=conn.cursor()
                
                #For Student Id
                my_cursor.execute("select Student_Id from student where Student_Id="+str(id))
                i=my_cursor.fetchone()
                i="+".join(i)
                
                # For Name
                my_cursor.execute("select Name from student where Student_Id="+str(id))
                n=my_cursor.fetchone()
                n="+".join(n)
                
                # For Roll Number
                my_cursor.execute("select Roll from student where Student_Id="+str(id))
                r=my_cursor.fetchone()
                r="+".join(r)
                
                #For Depertment
                my_cursor.execute("select Dep from student where Student_Id="+str(id))
                d=my_cursor.fetchone()
                d="+".join(d)
                
                if confidence>80:
                    cv2.putText(img, f"Id:{str(i)}",(x,y-75),cv2.FONT_HERSHEY_COMPLEX, 0.5,(255,255,255),2)
                    cv2.putText(img, f"Roll:{str(r)}",(x,y-55), cv2.FONT_HERSHEY_COMPLEX, 0.5,(255,255,255),2)
                    cv2.putText(img, f"Name:{str(n)}",(x,y-35),cv2.FONT_HERSHEY_COMPLEX, 0.5,(255,255,255),2)
                    cv2.putText(img, f"Depertment:{str(d)}",(x,y-10),cv2.FONT_HERSHEY_COMPLEX, 0.5,(255,255,255),2)
                    self.mark_attendance(i,r,n,d)
                else:
                    cv2.rectangle(img,(x,y),(x+w, y+h),(0,0,255),3)
                    cv2.putText(img, "Unknown Face",(x,y-55),cv2.FONT_HERSHEY_COMPLEX, 0.8,(255,255,255),3)
                
                coord=[x,y,w,y]
            return coord
        def recognize(img,clf,faceCascade):
            coord= draw_boundry(img,faceCascade,1.1,10,(255,25,255),"Face", clf)
            return img
        
        faceCascade=cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
        clf=cv2.face.LBPHFaceRecognizer_create()
        clf.read("classifier.xml")
        
        video_cap=cv2.VideoCapture(0)
        while True:
            ret,img=video_cap.read()
            img=recognize(img,clf,faceCascade)
            cv2.imshow("Welcome to Face Recognition", img)
                        
            if cv2.waitKey(1)==13:
                break
        video_cap.release()
        cv2.destroyAllWindows()            

In [15]:
if __name__=='__main__':
    root=Tk()
    obj = FaceReconizerSystem(root)
    root.mainloop()